In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [ ]:
# 当有本地csv数据时，使用以下方法获取数据
def csvLoader(csvpath):
    csv_data = pd.read_csv(csvpath,header=0)
    #print(csv_data)
    # 转换为numpy矩阵
    return np.array(csv_data)

PCA_projected_trainData = csvLoader('PCAProjectedTrainData800.csv')
PCA_projected_testData = csvLoader('PCAProjectedTestData800.csv')
#print(PCA_projected_trainData)

In [2]:
train_labels=[]
#with open(trainingFile, "r") as fr1:
#    trainFile = fr1.readlines()
def getListFromFile(_filename):
    _list = []
    with open(_filename, "r") as fr3:
        dat_r = fr3.readlines()
    for inputData in dat_r:
        _list.append(inputData[0])
    target = np.asarray(_list)
    return target

test_labels = getListFromFile('valid_labels.dat')
for c in getListFromFile('dorothea_train.labels'):
    if c!= '1':
        train_labels.append('0')
    else:train_labels.append(c)
train_labels=np.asarray(train_labels)
print(train_labels)

['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '1' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '1' '0' '1' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1'
 '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '1'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '1' '0' '1' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '1' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1' '0' '0' '1'
 '0' '0' '0' '0' '0' '1' '0' '1' '0' '0' '0' '0' '0

In [9]:
# Save data handled by PCA
def pcaData2csv(csvpath1,csvpath2, train_data,test_data,_sep=','):
    dftrain=pd.DataFrame(train_data)
    dftest=pd.DataFrame(test_data)
    dftrain.to_csv(csvpath1,index=False,sep=_sep)
    dftest.to_csv(csvpath2,index=False,sep=_sep)
    if os.path.exists(csvpath1):
        print("PCA projected train data has been written into csv files in:"+str(csvpath1))
        if os.path.exists(csvpath2):
            print("PCA projected test data has been written into csv files in:"+str(csvpath2))
        else:
            raise Warning("File written failed in:"+str(csvpath2))
    else:raise Warning("File written failed in:"+str(csvpath1))
    # 抛出dataframe类型以备其他用途
    return dftrain,dftest

In [10]:
train_labels2=train_labels.astype(int)
test_labels2=test_labels.astype(int)
path1=r"trainLables.csv"
path2=r"testLabels.csv"
#print(len(train_labels2),train_labels2)
# Also gotten by dorothea_train.labels→train_labels2(binary)
#print(len(test_labels2),test_labels2)
# Also gotten by dorothea_valid.labels→test_labels2(binary)
pcaData2csv(path1,path2, train_labels2,test_labels2)

PCA projected train data has been written into csv files in:trainLables.csv
PCA projected test data has been written into csv files in:testLabels.csv


(     0
 0    0
 1    0
 2    0
 3    0
 4    0
 ..  ..
 795  0
 796  0
 797  0
 798  0
 799  1
 
 [800 rows x 1 columns],
      0
 0    0
 1    0
 2    1
 3    0
 4    0
 ..  ..
 345  0
 346  1
 347  0
 348  0
 349  0
 
 [350 rows x 1 columns])

In [ ]:
#from sklearn.decomposition import PCA as sklearnPCA
from sklearn.metrics import roc_curve, roc_auc_score, recall_score
#Read the input files and read every line
def loadData(trainingFile, testingFile):
    
    def convertDataframe(inputFile):
        data = pd.DataFrame(columns=range(100000))
        
        for i in range(len(inputFile)):
            record = np.fromstring(inputFile[i], dtype=int, sep=' ')
            record_bool = [0 for j in range(100000)]
            for col in record:
                record_bool[col-1] = 1
            
            data.loc[i] = record_bool
            
        return data
    
    with open(trainingFile, "r") as fr1:
        trainFile = fr1.readlines()
    
    #Split each line in the two files into label and data  
    train_data_list = []
    train_labels_list = []
    
    for inputData in trainFile:
        train_labels_list.append(inputData[0])
        #Remove the activity label (0/1) and new line character from each record
        inputData = inputData.replace("0\t", "")
        inputData = inputData.replace("1\t", "")
        inputData = inputData.replace("\n", "")
        train_data_list.append(inputData)
    
    train_labels = np.asarray(train_labels_list)
    train_data = convertDataframe(train_data_list)
        
    with open(testingFile, "r") as fr2:
        testFile = fr2.readlines()
    
    test_data = convertDataframe(testFile)
            
    return train_data, test_data, train_labels




In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# Project data on a reduced dimensionality k using PCA
def pca(train_data, test_data, k):

    pca = sklearnPCA(n_components = k)
    PCA_projected_trainData = pca.fit_transform(train_data)
    PCA_projected_testData = pca.transform(test_data)
    
    return PCA_projected_trainData, PCA_projected_testData

def lda(train_data,train_labels, test_data,test_labels, k):
    # 创建线性判别分析对象，并指定降维后的维度为2
    lda = LinearDiscriminantAnalysis(n_components = k)

    # 对数据进行降维，并查看方差比例
    LDA_projected_trainData = lda.fit_transform(train_data,train_labels)
    LDA_projected_testData = lda.fit_transform(test_data,test_labels)
    print(lda.transform(test_data))
    print(lda.explained_variance_ratio_)
    return LDA_projected_trainData, LDA_projected_testData

In [ ]:
#Read the training and the test data set and get 3 separate dataframes of training reviews, test reviews and training labels
train_data, test_data, train_labels = loadData('train.dat', 'test.dat')

In [ ]:
#Reduce the number of dimensions from 100000 to 800 using PCA
train_labels = train_labels.astype(int)
test_labels = np.array(loadFile('valid_labels.dat')).astype(int)
PCA_projected_trainData, PCA_projected_testData = pca(train_data, test_data, 800)
LDA_projected_trainData, LDA_projected_testData = lda(train_data,train_labels, test_data,test_labels, 1)
print(PCA_projected_trainData, PCA_projected_testData)
print(LDA_projected_trainData, LDA_projected_testData)

In [ ]:
csv_path1=r"PCAProjectedTrainData800.csv"
csv_path2=r"PCAProjectedTestData800.csv"
pcaData2csv(csv_path1,csv_path2, PCA_projected_trainData,PCA_projected_testData)

In [ ]:
PCA_Data = np.vstack((PCA_projected_trainData,PCA_projected_testData))
all_labels = np.hstack((train_labels,test_labels))

In [ ]:
import matplotlib.pyplot as plt

threshold = np.max(LDA_projected_testData)-0.5*(np.max(LDA_projected_testData)-np.min(LDA_projected_testData))
pos_count = np.count_nonzero(LDA_projected_testData > threshold)
neg_count = np.count_nonzero(LDA_projected_testData < threshold)
print(pos_count, neg_count)

# 根据阈值设置颜色
colors = np.ravel(np.where(LDA_projected_testData > threshold, 'red', 'blue'))

# 绘制散点图
plt.scatter(LDA_projected_testData, np.zeros(np.size(LDA_projected_testData,0)), c=colors)
plt.axvline(threshold, color='k', linestyle='--') # 绘制阈值线
plt.xlabel('Value')
plt.show()

# 导入sklearn.metrics模块
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc

# 定义真实标签和预测标签
y_pred = []
for i in range(np.size(LDA_projected_testData,0)):
    if LDA_projected_testData[i,0]>threshold:
        y_pred.append(1)
    else:y_pred.append(0)
# 计算准确率
acc = accuracy_score(test_labels, y_pred)
print("Accuracy: {:.4f}".format(acc))

# 计算精确率
precision = precision_score(test_labels, y_pred)
print("Precision: {:.4f}".format(precision))

# 计算召回率
recall = recall_score(test_labels, y_pred)
print("Recall: {:.4f}".format(recall))

# 计算F1分数
f1 = f1_score(test_labels, y_pred)
print("F1-score: {:.4f}".format(f1))

# 计算ROC曲线和AUC
fpr, tpr, thresholds = roc_curve(test_labels, y_pred)
roc_auc = auc(fpr, tpr)
print("ROC curve: fpr = {}, tpr = {}, AUC = {:.4f}".format(fpr, tpr, roc_auc))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, average_precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
def resultAnalysis(testData,testLabels,pred,proba,_module=None):
    xmodulelist = ['macro','micro','weighted','non-avg']
    criteion = ['Accuracy','Precision','Recall','F1_score','AUC','AUPRC']
    df = pd.DataFrame(index=xmodulelist,columns=criteion)
    
    
    if _module not in xmodulelist:
        
        df['Accuracy']['non-avg']=round(accuracy_score(testLabels,pred),3)
        df['Precision']['non-avg']=round(precision_score(testLabels,pred,),3)
        df['Recall']['non-avg']=round(recall_score(testLabels,pred,),3)
        df['AUC']['non-avg']=round(roc_auc_score(testLabels,proba,),3)
        df['AUPRC']['non-avg']=round(average_precision_score(testLabels,pred,),3)
        df['F1_score']['non-avg']=round(f1_score(testLabels,pred,),3)
        print(df.iloc[3,:])
            
    else:
        df['Accuracy'][_module]=round(accuracy_score(testLabels,pred),3)
        df['Precision'][_module]=round(precision_score(testLabels,pred,average=_module),3)
        df['Recall'][_module]=round(recall_score(testLabels,pred,average=_module),3)
        df['AUC'][_module]=round(roc_auc_score(testLabels,proba,average=_module),3)
        df['AUPRC'][_module]=round(average_precision_score(testLabels,pred,average=_module),3)
        df['F1_score'][_module]=round(f1_score(testLabels,pred,average=_module),3)
        print(df.loc[_module,:])
    print(confusion_matrix(testLabels,pred))
    return classification_report(testLabels,pred)

In [ ]:
# BernoulliNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import auc

def BNBC2(trainData,trainLabels, testData,testLabels):
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    
    clf = BernoulliNB (alpha=1.0, binarize=0.0, fit_prior=True, class_prior=None)
    clf.fit(trainData,trainLabels)
    
    pred = clf.predict(testData)
    proba = clf.predict_proba(testData)[:,1]
    score = clf.score(testData,testLabels)
    fpr, tpr, thresholds = roc_curve(testLabels, proba)
    rocauc=auc(fpr, tpr)
    print(resultAnalysis(testData,testLabels,pred,proba,))
    #print(proba)
    print("\tAccuracy:{:.3f}".format(score))
    print("\tPrecision:{:.3f}".format(precision_score(testLabels,pred)))
    print("\tRecall:{:.3f}".format(recall_score(testLabels,pred)))
    print("\tF1_score:{:.3f}".format(f1_score(testLabels,pred)))
    print("\tAUC:{:.3f}".format(roc_auc_score(testLabels,proba)))
    return pred

BNBC2(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels)

In [ ]:
from sklearn.naive_bayes import ComplementNB
from sklearn.preprocessing import MinMaxScaler
 
def min_max_normalization(np_array):
    min_max_scaler = MinMaxScaler(feature_range=[0,10])
    ret = min_max_scaler.fit_transform(np_array)
    return ret

def CNB(trainData,trainLabels, testData,testLabels):

    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    
    trainData=min_max_normalization(trainData)
    testData=min_max_normalization(testData)
    
    clf = ComplementNB(alpha=1.0, fit_prior=True, class_prior=None, norm=False)
    clf.fit(trainData,trainLabels)

    pred = clf.predict(testData)
    proba = clf.predict_proba(testData)[:,1]
    score = clf.score(testData,testLabels)
    fpr, tpr, thresholds = roc_curve(testLabels, proba)
    #rocauc=auc(fpr, tpr)
    print(resultAnalysis(testData,testLabels,pred,proba,))
    # 以下针对少数类
    print("\tAccuracy:{:.3f}".format(score))
    print("\tPrecision:{:.3f}".format(precision_score(testLabels,pred)))
    print("\tRecall:{:.3f}".format(recall_score(testLabels,pred)))
    print("\tAUC:{:.3f}".format(roc_auc_score(testLabels,proba)))
    return pred

CNB(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels)

In [ ]:
#SVM

from sklearn import svm
#from sklearn.metrics import roc_curve, roc_auc_score, recall_score, precision_score
#from imblearn import specificity_score

def SVMC_kernel(trainData,trainLabels, testData,testLabels, weights='balanced'):
    #clf = svm.SVC(gamma='scale', class_weight=weights)
    #clf.fit(trainData,trainLabels)
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    for k in ['linear','rbf','poly','sigmoid']:
        clf_proba = svm.SVC(kernel=k,C=100,probability=True).fit(trainData,trainLabels)
        #print(clf_proba.decision_function(testData).shape)
        #predictions = clf.predict(testData)
        clf_pred = clf_proba.predict(testData)
        result = clf_proba.score(testData, testLabels)
        score = clf_proba.decision_function(testData)

        rocauc = roc_auc_score(testLabels, score)
        recall = recall_score(testLabels, clf_pred)
        precision = precision_score(testLabels, clf_pred)
        print("The kernel {} prediction is:".format(k))
        print(resultAnalysis(testData,testLabels,clf_pred,score,))
    #print("\tAccuracy:{:.3f}".format(result))
    #print("\tPrecision:{:.3f}".format(precision))
    #print("\tRecall:{:.3f}".format(recall))
    #print("\tSpecificity:{:.3f}".format(specificity_score(testLabels, clf_pred)))
    #print("\tAUC:{:.3f}".format(rocauc))
    #return clf_pred

SVMC_kernel(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels)

In [ ]:
def SVMC(trainData,trainLabels, testData,testLabels,weights='balanced'):
    
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    
    clf_proba = svm.SVC(gamma='scale', class_weight=weights).fit(trainData,trainLabels)
    #print(clf_proba.decision_function(testData).shape)
    #predictions = clf.predict(testData)
    
    clf_pred = clf_proba.predict(testData)
    
    result = clf_proba.score(testData, testLabels)
    score = clf_proba.decision_function(testData)
    
    rocauc = roc_auc_score(testLabels, score)
    recall = recall_score(testLabels, clf_pred)
    precision = precision_score(testLabels, clf_pred)
    
    print(resultAnalysis(testData,testLabels,clf_pred,score,))
    #print("\tAccuracy:{:.3f}".format(result))
    #print("\tPrecision:{:.3f}".format(precision))
    #print("\tRecall:{:.3f}".format(recall))
    #print("\tSpecificity:{:.3f}".format(specificity_score(testLabels, clf_pred)))
    #print("\tAUC:{:.3f}".format(rocauc))
    return clf_pred
SVMC(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels)

In [ ]:
from sklearn.ensemble import RandomForestClassifier     #随机森林

def RFC(trainData,trainLabels, testData,testLabels):
    
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    
    rfc = RandomForestClassifier(n_estimators=63, max_depth=20,max_features=26,min_samples_leaf=3,min_samples_split=14,criterion='entropy',
                                 class_weight='balanced',random_state=42)                      #不调参
    rfc = rfc.fit(trainData, trainLabels)                 #用训练集数据训练模型
    pred = rfc.predict(testData)
    result = rfc.score(testData, testLabels)
    proba = rfc.predict_proba(testData)[:,1]
    rocauc = roc_auc_score(testLabels, proba)
    recall = recall_score(testLabels, pred)
    precision = precision_score(testLabels, pred)
    print(resultAnalysis(testData,testLabels,pred,proba,))
    
    return pred
RFC(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels)

In [ ]:
from sklearn.ensemble import RandomForestClassifier     #随机森林
def RFC2(trainData,trainLabels, testData,testLabels,paramdict):
    
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)#二值化
    #paramdict={'n_estimators':63, 'max_depth':5,'criterion':'entropy','class_weight':'balanced','random_state':42}
    rfc = RandomForestClassifier(n_estimators=paramdict['n_estimators'], 
                                 max_features=paramdict['max_features'],
                                 min_samples_split=paramdict['min_samples_split'],
                                 min_samples_leaf=paramdict['min_samples_leaf'], 
                                 min_weight_fraction_leaf=paramdict['min_weight_fraction_leaf'],
                                 criterion=paramdict['criterion'],
                                 class_weight='balanced',random_state=paramdict['random_state']) 
    #rfc = RandomForestClassifier(n_estimators=63, max_depth=5,criterion='entropy',class_weight='balanced',random_state=42)
    start = time()
    rfc = rfc.fit(trainData, trainLabels)                 #用训练集数据训练模型
    end = time()
    elapsed = end - start
    print("Time to train model RF: %.9f seconds" % elapsed)
    pred = rfc.predict(testData)
    result = rfc.score(testData, testLabels)
    proba = rfc.predict_proba(testData)[:,1]
    rocauc = roc_auc_score(testLabels, proba)
    recall = recall_score(testLabels, pred)
    precision = precision_score(testLabels, pred)
    print(resultAnalysis(testData,testLabels,pred,proba,))
    
    return pred
#GS.fit(PCA_projected_trainData, train_labels)
#print(GS.best_params_)
#print(GS.best_score_)

parameters={'n_estimators':63, 'max_depth':20,'max_features':26,'min_samples_split':14, 'min_samples_leaf':3,'min_weight_fraction_leaf':0.00,
            'criterion':'entropy','class_weight':'balanced','random_state':42}
RFC2(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels,parameters)

In [ ]:
import xgboost as xgb
from time import *

#def load_csv(path):
#    df = pd.read_csv(path)
#    target = df['cls']
#    df = df.drop(['cls'], axis=1)
#    return xgb.DMatrix(df.values, label=target.values)

#dtest = load_csv('/hdd/hdd1/twonormData/t1000.csv')
#xgb.DMatrix(PCA_projected_trainData, label=train_labels.astype(int))
def evaluate(trainData,trainLabels, testData,testLabels, num_trees=100,max_depth=50,num_jobs=-1):
    trainLabels=trainLabels.astype(int)
    testLabels=testLabels.astype(int)
    dtrain = xgb.DMatrix(trainData, label=trainLabels)
    dtest = xgb.DMatrix(testData, label=testLabels)
    param = {'num_parallel_tree':num_trees, 'max_depth':max_depth, 'objective':'binary:logistic',
        'nthread':num_jobs, 'subsample':0.7,'scale_pos_weight':9}
    start = time()
    model = xgb.train(param, dtrain, 1)
    # 性能评估以XGboost为例
    #xgb = xgb.XGBClassifier()
    # 对训练集训练模型
    #xgb.fit(trainData,trainLabels)
    # 对测试集进行预测
    #y_pred = xgb.predict(testData)
    end = time()
    elapsed = end - start
    print("Time to train XGBoost model: %.9f seconds" % elapsed)
    prediction = model.predict(dtest)
    #proba = model.predict_proba(testData)[:,1]
    #print(prediction.shape)
    length=len(prediction)
    proba = np.stack([[1-a for a in prediction],prediction],axis=-1)
    pred=[]
    for i in range(len(prediction)):
        if prediction[i] >0.5:
            pred.append(1)
        else:pred.append(0)
    
    #print(pred)
    #print("Accuracy = %.3f" % np.mean(prediction == dtest.get_label()))
    print(resultAnalysis(testData,testLabels,pred,prediction))

evaluate(PCA_projected_trainData,train_labels, PCA_projected_testData,test_labels,) 

调参是一个非常重要的步骤，可以帮助我们优化机器学习算法的表现。对于sklearn随机森林算法，调参需要注意以下几点：

n_estimators：随机森林中树的数量。增加这个参数可以提高算法的表现，但同时会使得算法的计算时间变长。因此，需要根据具体情况进行调整。

max_depth：树的最大深度。增加这个参数可以提高算法的表现，但同时也会使得算法的计算时间变长。需要注意的是，如果max_depth设置太大，会导致过拟合的问题。

min_samples_split：节点进行分裂所需的最小样本数量。增加这个参数可以防止过拟合，但会使得算法的表现变差。

min_samples_leaf：叶节点所需的最小样本数量。增加这个参数可以防止过拟合，但会使得算法的表现变差。

max_features：特征随机选择的个数。增加这个参数可以提高算法的表现，但也会增加算法的计算时间。

bootstrap：是否有放回地进行采样。如果设置为False，会使得随机森林变成极端随机森林。

criterion：衡量分裂质量的准则。一般选择“gini”或“entropy”。

## Grid Search-网格搜索
调参的过程可以使用网格搜索(Grid Search)或随机搜索(Random Search)等方法进行。可以通过如下代码进行调参：

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier()

# 设置要调整的参数
parameters = {'n_estimators': [10, 50, 100],
              'max_depth': [1, 5, 10],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2],
              'max_features': [2, 4],
              'bootstrap': [True, False],
              'criterion': ['gini', 'entropy']}

# 使用网格搜索进行调参
grid_search = GridSearchCV(clf, parameters, cv=3, n_jobs=1, verbose=1)
grid_search.fit(X, y)

# 输出最佳的参数组合和得分
print(grid_search.best_params_)
print(grid_search.best_score_)
```
在网格搜索的过程中，我们首先创建了一个空的随机森林分类器，然后设置了要调整的参数。我们还设置了交叉验证的次数(cv)和并行处理的数量(n_jobs)。然后我们使用GridSearchCV函数进行网格搜索，并使用了verbose参数指定详细程度。最后输出最佳的参数组合和得分。

## 贝叶斯优化
scikit-learn里的内置贝叶斯优化库为BayesSearchCV，该库可以在超参数空间中使用朴素贝叶斯优化算法，以尝试找到最佳超参数组合。BayesSearchCV需要设定要优化的超参数的搜索范围和目标指标，并为算法提供模型评估器。

下面是一个简单的随机森林模型的贝叶斯优化调参示例代码：

```python
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer

# 定义随机森林模型并设定超参数空间
rf = RandomForestClassifier(n_estimators=100, random_state=0)
param_dist = {'max_depth': Integer(10, 50),
              'min_samples_split': Integer(2, 10),
              'min_samples_leaf': Integer(1, 10),
              'max_features': Real(0.1, 1.0)}

# 使用交叉验证来对模型进行评估
def objective_func(params):
    rf.set_params(**params)
    return -np.mean(cross_val_score(rf, X, y, cv=5, scoring='f1'))

# 在超参数空间上运用贝叶斯搜索来优化模型
opt = BayesSearchCV(rf,
                    param_dist,
                    n_iter=25,
                    scoring='f1',
                    cv=5,
                    random_state=0)
opt.fit(X, y)
```
在上面的代码中，使用了sklearn的内置贝叶斯优化库BayesSearchCV。首先定义了一个随机森林模型并设定了超参数空间，然后使用交叉验证评估模型的表现，并将评估结果作为目标函数。最后，在超参数空间上使用BayesSearchCV进行参数搜索，搜索结束后可以使用如下代码来输出参数搜索的结果：

```python
print("Best F1 Score:3f" opt.best_score_)
print("Best Parameters:s" str(opt.best_params_))
```
使用BayesSearchCV时，可以通过参数n_iter来控制贝叶斯优化的迭代次数，通常n_iter要比随机搜索大，这样可以更好地探索超参数空间。同时，参数cv指定了用于交叉验证的数据集的拆分数量，一般默认为5。scoring参数用于指定评估指标，这里选择了F1得分，因为它同时考虑了准确率和召回率，是一个比较全面的指标。

需要注意的是，由于贝叶斯优化是一种基于概率的优化方法，因此它的寻优速度可能会受到几个因素的影响，例如超参数的数量和范围、目标函数的复杂性等。因此，在进行调参时，需要根据实际情况和计算资源情况进行适当的调整和控制。

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
#from functools import partial
from skopt import BayesSearchCV
from skopt.space import Real, Integer

X=PCA_projected_trainData
y=train_labels.astype(int)

# 定义随机森林模型并设定超参数空间
rf = RandomForestClassifier(random_state=42)
param_dist = {'max_depth': Integer(1, 20),
              'n_estimators': Integer(100, 250),
              'min_samples_split': Integer(2, 10),
              'min_samples_leaf': Integer(1, 10),
              'min_weight_fraction_leaf': Real(0, 0.5),
              'criterion': ['gini', 'entropy'],
              }

# 使用交叉验证来对模型进行评估
def objective_func(params):
    rf.set_params(**params)
    return -np.mean(cross_val_score(rf, X, y, cv=10, scoring='f1'))

# 在超参数空间上运用贝叶斯搜索来优化模型
opt = BayesSearchCV(rf,
                    param_dist,
                    n_iter=35,
                    scoring='recall',
                    cv=10,
                    random_state=42)
opt.fit(X, y)
print("Best F1 Score:%3f" % opt.best_score_)
print("Best Parameters:%s" % str(opt.best_params_))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
# 调参，绘制学习曲线来调参n_estimators（对随机森林影响最大）
score_lt = []

# 每隔10步建立一个随机森林，获得不同n_estimators的得分
for i in range(50,200,10):
    rfc = RandomForestClassifier(n_estimators=i+1,random_state=42)
    score = cross_val_score(rfc, PCA_Data, all_labels.astype(int), cv=10,scoring='roc_auc').mean()
    score_lt.append(score)
score_max = max(score_lt)
print('最大得分：{}'.format(score_max),
      '子树数量为：{}'.format(score_lt.index(score_max)*10+1))

# 绘制学习曲线
x = np.arange(51,201,10)
plt.subplot(111)
plt.plot(x, score_lt, 'r-')
plt.show()